# Add yolo results to dataframe, compute od in original image, save images 

In [11]:
import pandas as pd
import os
from tqdm import tqdm
from collections import Counter
import src.preprocessing as ppc
import cv2

In [12]:
df = pd.read_csv("data/shuffled_square_75/img_info.csv", index_col=0)
df

,new_file,delta_x,delta_y,orig_crop_side,side,scaling
orig_file,,,,,,
DEV13781.jpg,SHUF00000,176,-144,2272,2272,1.0
DEV12858.jpg,SHUF00001,-89,-904,3150,3150,1.0
DEV02899.jpg,SHUF00002,-59,-685,2486,2486,1.0
DEV12129.jpg,SHUF00003,168,-152,2256,2256,1.0
DEV04490.jpg,SHUF00004,-66,-886,3166,3166,1.0
...,...,...,...,...,...,...
DEV04506.jpg,SHUF14995,-44,-905,3175,3175,1.0
DEV12149.jpg,SHUF14996,-45,-857,3207,3207,1.0
DEV00409.jpg,SHUF14997,-95,-870,2987,2987,1.0


## Extract predictions and add to dataframe

In [13]:
leng = []

for filename in tqdm(os.listdir("data/labels")):
    predictions = open(f"data/labels/{filename}", "r").read()
    predictions = predictions.split("\n")[:-1]
    predictions = [pred.split() for pred in predictions]
    leng.append(len(predictions))

100%|██████████| 14999/14999 [00:00<00:00, 57638.31it/s]


In [14]:
count = Counter(leng)
count # in 117 instances we only have 1 prediction 
# (I checked, its like half/half fovea or OD. so like 50 ODs are missing but in that case we can just use the full retina)

Counter({2: 14709, 1: 188, 3: 100, 4: 2})

In [15]:
len(os.listdir("data/labels"))

14999

In [16]:
records = []
for filename in tqdm(os.listdir("data/labels")):
    key = filename[:-4]
    with open(f"data/labels/{filename}", "r") as f:
        odc = [None] * 5
        fovea = [None] * 5
        for line in f.readlines():
            pred = [float(number) for number in line.split()]
            if pred[0]==0:
                odc=pred[1:]
            if pred[0]==1:
                fovea=pred[1:]

        record = [key,]
        record.extend(odc)
        record.extend(fovea)
        records.append(tuple(record))

df_to_join = pd.DataFrame.from_records(records, columns=[
    'new_file',
    'odc_x_ratio', 'odc_y_ratio', 'odc_width_ratio', 'odc_height_ratio', 'odc_conf',
    'fovea_x_ratio', 'fovea_y_ratio', 'fovea_width_ratio', 'fovea_height_ratio', 'fovea_conf'])

max_fovea = df_to_join.fovea_conf.max()
max_odx = df_to_join.odc_conf.max()
df = pd.merge(df.reset_index(), df_to_join, how='left', on='new_file')
df

100%|██████████| 14999/14999 [00:00<00:00, 52587.89it/s]


,orig_file,new_file,delta_x,delta_y,orig_crop_side,side,scaling,odc_x_ratio,odc_y_ratio,odc_width_ratio,odc_height_ratio,odc_conf,fovea_x_ratio,fovea_y_ratio,fovea_width_ratio,fovea_height_ratio,fovea_conf
0,DEV13781.jpg,SHUF00000,176,-144,2272,2272,1.0,0.789844,0.457812,0.192188,0.187500,0.882039,0.392969,0.501562,0.207813,0.212500,0.807168
1,DEV12858.jpg,SHUF00001,-89,-904,3150,3150,1.0,0.238281,0.442969,0.198437,0.201562,0.876956,0.612500,0.526563,0.187500,0.196875,0.793723
2,DEV02899.jpg,SHUF00002,-59,-685,2486,2486,1.0,0.304688,0.441406,0.162500,0.164062,0.894259,0.670313,0.506250,0.159375,0.162500,0.755753
3,DEV12129.jpg,SHUF00003,168,-152,2256,2256,1.0,0.254687,0.442969,0.156250,0.157813,0.888492,0.608594,0.485938,0.164062,0.162500,0.737824
4,DEV04490.jpg,SHUF00004,-66,-886,3166,3166,1.0,0.802344,0.489844,0.201562,0.201562,0.881505,0.450000,0.534375,0.193750,0.196875,0.843570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,DEV04506.jpg,SHUF14995,-44,-905,3175,3175,1.0,0.235156,0.472656,0.154687,0.151562,0.900382,0.611719,0.531250,0.167187,0.171875,0.852776
14996,DEV12149.jpg,SHUF14996,-45,-857,3207,3207,1.0,0.257031,0.489844,0.192188,0.195312,0.876265,0.601562,0.518750,0.196875,0.200000,0.831810
14997,DEV00409.jpg,SHUF14997,-95,-870,2987,2987,1.0,0.625781,0.466406,0.195312,0.192188,0.870469,0.220313,0.532031,0.200000,0.207813,0.828260
14998,DEV01824.jpg,SHUF14998,197,-127,2338,2338,1.0,0.753906,0.458594,0.151562,0.157813,0.905078,0.393750,0.503125,0.153125,0.153125,0.795078


## Calculate OD size and center on original image

In [17]:
df["fovea_x_square"] = round(df.orig_crop_side * df.fovea_x_ratio, 0)
df["odc_x_square"] = round(df.orig_crop_side * df.odc_x_ratio, 0)
df["fovea_y_square"] = round(df.orig_crop_side * df.fovea_y_ratio, 0)
df["odc_y_square"] = round(df.orig_crop_side * df.odc_y_ratio, 0)
df["od_side_ratio_avg"] = (df.odc_height_ratio + df.odc_width_ratio)/2
df["odc_side_pxl"] = round(df.orig_crop_side * df.od_side_ratio_avg, 0)
df["odc_x_rect"] = df.odc_x_square + df.delta_x
df["odc_y_rect"] = df.odc_y_square + df.delta_y

df

,orig_file,new_file,delta_x,delta_y,orig_crop_side,side,scaling,odc_x_ratio,odc_y_ratio,odc_width_ratio,...,fovea_height_ratio,fovea_conf,fovea_x_square,odc_x_square,fovea_y_square,odc_y_square,od_side_ratio_avg,odc_side_pxl,odc_x_rect,odc_y_rect
0,DEV13781.jpg,SHUF00000,176,-144,2272,2272,1.0,0.789844,0.457812,0.192188,...,0.212500,0.807168,893.0,1795.0,1140.0,1040.0,0.189844,431.0,1971.0,896.0
1,DEV12858.jpg,SHUF00001,-89,-904,3150,3150,1.0,0.238281,0.442969,0.198437,...,0.196875,0.793723,1929.0,751.0,1659.0,1395.0,0.199999,630.0,662.0,491.0
2,DEV02899.jpg,SHUF00002,-59,-685,2486,2486,1.0,0.304688,0.441406,0.162500,...,0.162500,0.755753,1666.0,757.0,1259.0,1097.0,0.163281,406.0,698.0,412.0
3,DEV12129.jpg,SHUF00003,168,-152,2256,2256,1.0,0.254687,0.442969,0.156250,...,0.162500,0.737824,1373.0,575.0,1096.0,999.0,0.157031,354.0,743.0,847.0
4,DEV04490.jpg,SHUF00004,-66,-886,3166,3166,1.0,0.802344,0.489844,0.201562,...,0.196875,0.843570,1425.0,2540.0,1692.0,1551.0,0.201562,638.0,2474.0,665.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,DEV04506.jpg,SHUF14995,-44,-905,3175,3175,1.0,0.235156,0.472656,0.154687,...,0.171875,0.852776,1942.0,747.0,1687.0,1501.0,0.153124,486.0,703.0,596.0
14996,DEV12149.jpg,SHUF14996,-45,-857,3207,3207,1.0,0.257031,0.489844,0.192188,...,0.200000,0.831810,1929.0,824.0,1664.0,1571.0,0.193750,621.0,779.0,714.0
14997,DEV00409.jpg,SHUF14997,-95,-870,2987,2987,1.0,0.625781,0.466406,0.195312,...,0.207813,0.828260,658.0,1869.0,1589.0,1393.0,0.193750,579.0,1774.0,523.0
14998,DEV01824.jpg,SHUF14998,197,-127,2338,2338,1.0,0.753906,0.458594,0.151562,...,0.153125,0.795078,921.0,1763.0,1176.0,1072.0,0.154688,362.0,1960.0,945.0


In [18]:
df.to_csv("data/img_info_extended.csv")

In [19]:
df = pd.read_csv("data/img_info_extended.csv", index_col=0)
df

,orig_file,new_file,delta_x,delta_y,orig_crop_side,side,scaling,odc_x_ratio,odc_y_ratio,odc_width_ratio,...,fovea_height_ratio,fovea_conf,fovea_x_square,odc_x_square,fovea_y_square,odc_y_square,od_side_ratio_avg,odc_side_pxl,odc_x_rect,odc_y_rect
0,DEV13781.jpg,SHUF00000,176,-144,2272,2272,1.0,0.789844,0.457812,0.192188,...,0.212500,0.807168,893.0,1795.0,1140.0,1040.0,0.189844,431.0,1971.0,896.0
1,DEV12858.jpg,SHUF00001,-89,-904,3150,3150,1.0,0.238281,0.442969,0.198437,...,0.196875,0.793723,1929.0,751.0,1659.0,1395.0,0.199999,630.0,662.0,491.0
2,DEV02899.jpg,SHUF00002,-59,-685,2486,2486,1.0,0.304688,0.441406,0.162500,...,0.162500,0.755753,1666.0,757.0,1259.0,1097.0,0.163281,406.0,698.0,412.0
3,DEV12129.jpg,SHUF00003,168,-152,2256,2256,1.0,0.254687,0.442969,0.156250,...,0.162500,0.737824,1373.0,575.0,1096.0,999.0,0.157031,354.0,743.0,847.0
4,DEV04490.jpg,SHUF00004,-66,-886,3166,3166,1.0,0.802344,0.489844,0.201562,...,0.196875,0.843570,1425.0,2540.0,1692.0,1551.0,0.201562,638.0,2474.0,665.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,DEV04506.jpg,SHUF14995,-44,-905,3175,3175,1.0,0.235156,0.472656,0.154687,...,0.171875,0.852776,1942.0,747.0,1687.0,1501.0,0.153124,486.0,703.0,596.0
14996,DEV12149.jpg,SHUF14996,-45,-857,3207,3207,1.0,0.257031,0.489844,0.192188,...,0.200000,0.831810,1929.0,824.0,1664.0,1571.0,0.193750,621.0,779.0,714.0
14997,DEV00409.jpg,SHUF14997,-95,-870,2987,2987,1.0,0.625781,0.466406,0.195312,...,0.207813,0.828260,658.0,1869.0,1589.0,1393.0,0.193750,579.0,1774.0,523.0
14998,DEV01824.jpg,SHUF14998,197,-127,2338,2338,1.0,0.753906,0.458594,0.151562,...,0.153125,0.795078,921.0,1763.0,1176.0,1072.0,0.154688,362.0,1960.0,945.0


## Crop original images

In [20]:
FACTOR = 1.5
THRESHOLD = 10
MULTIPROCESSING_WORKERS = 8
RESOLUTION = 224
INTERPOLATION_METHOD = cv2.INTER_LINEAR
DATA_DIR = f'./data/ods_center_{FACTOR}_{RESOLUTION}_{INTERPOLATION_METHOD}'
if not os.path.isdir(DATA_DIR):
    print(f'{DATA_DIR} does not exist, creating dir')
    os.mkdir(DATA_DIR)

def process_file(filename):
    img = cv2.imread(f"data/cfp/{filename}")
    img_data = df.loc[df.orig_file == filename]
    write_filename = f'{img_data.new_file.values[0]}.png'
    if img_data.odc_x_rect.isna().values[0]:
        # TODO: This is not correct, a separate model should be trained for images without an optic disc being detected
        img_square, _, _ = ppc.make_square(img, THRESHOLD)
        img_res = ppc.resize_square(img_square, RESOLUTION)
        cv2.imwrite(f"{DATA_DIR}/{write_filename}", img_res)
    else:
        add_top = (img.shape[0] - img_data.side)/2
        add_left = (img.shape[1] - img_data.side)/2
        odc_x_rect = int(img_data.odc_x_square.values[0] + add_left)
        odc_y_rect = int(img_data.odc_y_square.values[0] + add_top)

        #od = ppc.crop_od(img, odc_x_rect, odc_y_rect, int(img_data.odc_side_pxl.values[0]*FACTOR))
        od = ppc.crop_od_fill_if_needed(img, odc_x_rect, odc_y_rect, int(img_data.odc_side_pxl.values[0]*FACTOR))

        od_res = ppc.resize_square(od, RESOLUTION)
        #print(write_filename)
        cv2.imwrite(f"{DATA_DIR}/{write_filename}", od_res) # png because it is lossless

cfp_files = os.listdir("data/cfp")
if MULTIPROCESSING_WORKERS > 1:
    from multiprocessing import Pool
    with Pool(MULTIPROCESSING_WORKERS) as pool:
        op_metadata = list(tqdm(pool.imap(process_file, cfp_files), total=len(cfp_files)))
    print('Finished.')
else:
    for filename in tqdm(os.listdir("data/cfp")):
        process_file(filename)

100%|██████████| 15000/15000 [02:16<00:00, 109.86it/s]

Finished.
